# prospect + specutils

Project: Convert DESI, SDSS & eBOSS spectra to specutils container objects, and modify prospect to accept those objects.

In [ ]:
import os
from specutils import Spectrum1D, SpectrumCollection, SpectrumList

## DESI spectra file

DESI spectra are stored by "channel" ("channel", "band" and "spectrograph arm" are used interchangably).  There are 10 spectrographs, each with three arms, called 'b', 'r' & 'z'.  In a DESI spectra file, all arms are grouped together, with a common wavelength solution for each arm, but the solutions do not overlap or have the same shape.  Thus we can't use a Spectrum1D or SpectrumCollection object, but we can use a SpectrumList containing three Spectrum1D objects.

In [1]:
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.environ['DESI_ROOT'], 'datachallenge', 'reference_runs', '19.12', 'spectro', 'redux')
os.environ['SPECPROD'] = 'mini'
nside = '64'
pixnum = '5305'
pixgroup = pixnum[0:2]
desi_spectra = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'],
                            f'spectra-{nside}', pixgroup, pixnum, f'spectra-{nside}-{pixnum}.fits')
desi_redshifts = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'],
                              f'spectra-{nside}', pixgroup, pixnum, f'zbest-{nside}-{pixnum}.fits')
print(desi_spectra)
print(desi_redshifts)

/Users/weaver/Documents/Data/desi/datachallenge/reference_runs/19.12/spectro/redux/mini/spectra-64/53/5305/spectra-64-5305.fits
/Users/weaver/Documents/Data/desi/datachallenge/reference_runs/19.12/spectro/redux/mini/spectra-64/53/5305/zbest-64-5305.fits


## SDSS/eBOSS spPlate file

SDSS spectra are stored per-plate in spPlate files.  These contain 640 spectra for the original SDSS spectrograph or 1000 spectra for the BOSS/eBOSS spectrograph.  All spPlate files have a common wavelength solution, so a spPlate file can be represented by a Spectrum1D object.

In [2]:
os.environ['SPECTRO_REDUX'] = os.path.join(os.environ['HOME'], 'Documents', 'Data', 'sdss', 'dr16', 'sdss', 'spectro', 'redux')
run2d = '26'
plate = '2955'
mjd = '54526'
sdss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
sdss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spZbest-{plate}-{mjd}.fits')
print(sdss_spectra)
print(sdss_redshifts)

/Users/weaver/Documents/Data/sdss/dr16/sdss/spectro/redux/26/2955/spPlate-2955-54526.fits
/Users/weaver/Documents/Data/sdss/dr16/sdss/spectro/redux/26/2955/spZbest-2955-54526.fits


In [3]:
run2d = 'v5_13_0'
plate = '9599'
mjd = '58131'
eboss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
eboss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, run2d, f'spZbest-{plate}-{mjd}.fits')
print(eboss_spectra)
print(eboss_redshifts)

/Users/weaver/Documents/Data/sdss/dr16/sdss/spectro/redux/v5_13_0/9599/spPlate-9599-58131.fits
/Users/weaver/Documents/Data/sdss/dr16/sdss/spectro/redux/v5_13_0/9599/v5_13_0/spZbest-9599-58131.fits


In [ ]:
for f in (desi_spectra, desi_redshifts, sdss_spectra, sdss_redshifts, eboss_spectra, eboss_redshifts);
    assert os.path.exists(f)